<h3>HTTP requests to the NASA Kennedy Space Center WWW server </h3> 
 
<p>Fonte oficial do dateset: http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html 
Dados:</p>
<ul>
<li>Jul 01 to Jul 31, ASCII format, 20.7 MB gzip compressed, 205.2 MB. </li>
<li>Aug 04 to Aug 31, ASCII format, 21.8 MB gzip compressed, 167.8 MB. </li> 
</ul>

<p><b>Sobre o dataset:</b> Esses dois conjuntos de dados possuem todas as requisições HTTP para o servidor da NASA Kennedy Space Center WWW na Flórida para um período específico. 
Os logs estão em arquivos ASCII com uma linha por requisição com as seguintes colunas: </p> 
<ul>
<li> <b>Host fazendo a requisição. </b> Um hostname quando possível, caso contrário o endereço de internet se o nome não puder ser identificado. </li> 
<li> <b>Timestamp</b> no formato "DIA/MÊS/ANO:HH:MM:SS TIMEZONE" </li>
<li> <b>Requisição</b> (entre aspas) </li>
<li> <b>Código do retorno HTTP</b>  </li>
<li> <b>Total de bytes retornados </b></li> 
</ul>    

In [1]:
from datetime import datetime
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import isnan
from pyspark.sql.functions import udf, to_date, to_utc_timestamp
from pyspark.sql import functions as F
import pandas as pd

In [2]:
#CASO NECESSÁRIO
#!pip install --upgrade wget

In [4]:
raw_data_files = glob.glob('*.gz')
raw_data_files

['NASA_access_log_Aug95.gz',
 'v0.2.20.tar.gz',
 'NASA_access_log_Jul95.gz',
 'NASA_access_log_Aug95 (1).gz',
 'NASA_access_log_Jul95 (1).gz']

In [5]:
import wget
import glob

#Em caso de ainda não tiver o dataset
#Obtendo os dados 
#data1 = 'ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz'
#data2 = 'ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz'
#file1 = wget.download(data1)
#file2 = wget.download(data2)

#Caso já tenha o data set
arquivos_gz = glob.glob('*.gz')


In [6]:
#Arquivos .gz
arquivos_gz

['NASA_access_log_Aug95.gz',
 'v0.2.20.tar.gz',
 'NASA_access_log_Jul95.gz',
 'NASA_access_log_Aug95 (1).gz',
 'NASA_access_log_Jul95 (1).gz']

In [8]:
file1 = arquivos_gz[2] #NASA_access_log_Jul95.gz'
file2 = arquivos_gz[0] #NASA_access_log_Aug95.gz'

#Criando vertor de arquivos
file = [file1,file2]

In [9]:
spark = SparkSession.builder.getOrCreate()

#criando dataframe
df = spark.read.text(file)
#Criando RDD
df_rdd = df.rdd
df.take(10)

[Row(value='199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'),
 Row(value='unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0'),
 Row(value='199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0'),
 Row(value='burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0'),
 Row(value='205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985'),
 Row(value='d104.aa.net - - [01/Ju

In [10]:
def extraindo_dados(data):
        '''Esta método será responsável por gerar um dataframe com a separação correta dos dados de acordo com as equações regulares'''
        
        padrao_host = r'(^\S+\.[\S+\.]+\S+)\s'
        padrao_timestamp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
        padrao_end =  r'^.*"\w+\s+([^\s]+)\s+HTTP.*"'
        padrao_status = r'\s(\d{3})\s'
        padrao_tamanho_cont = r'\s(\d+)$'
        df_logs = data.select(regexp_extract('value', padrao_host, 1).alias('host'),
                         regexp_extract('value', padrao_timestamp, 1).alias('timestamp'),
                         regexp_extract('value', padrao_end, 1).alias('caminho'),
                         regexp_extract('value', padrao_status, 1).cast('integer').alias('status'),
                         regexp_extract('value', padrao_tamanho_cont, 1).cast('integer').alias('tam_conteudo'))
        return df_logs


In [11]:
df_logs = extraindo_dados(df)
df_logs.show()

+--------------------+--------------------+--------------------+------+------------+
|                host|           timestamp|             caminho|status|tam_conteudo|
+--------------------+--------------------+--------------------+------+------------+
|        199.72.81.55|01/Jul/1995:00:00...|    /history/apollo/|   200|        6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00...| /shuttle/countdown/|   200|        3985|
|      199.120.110.21|01/Jul/1995:00:00...|/shuttle/missions...|   200|        4085|
|  burger.letters.com|01/Jul/1995:00:00...|/shuttle/countdow...|   304|           0|
|      199.120.110.21|01/Jul/1995:00:00...|/shuttle/missions...|   200|        4179|
|  burger.letters.com|01/Jul/1995:00:00...|/images/NASA-logo...|   304|           0|
|  burger.letters.com|01/Jul/1995:00:00...|/shuttle/countdow...|   200|           0|
|     205.212.115.106|01/Jul/1995:00:00...|/shuttle/countdow...|   200|        3985|
|         d104.aa.net|01/Jul/1995:00:00...| /shuttle/countdown/| 

In [13]:
#Permite usar comandos SQL para trabalhar com spark dataframe
df_logs.createOrReplaceTempView("logs")
spark.sql("SELECT * FROM logs").take(5)

[Row(host='199.72.81.55', timestamp='01/Jul/1995:00:00:01 -0400', caminho='/history/apollo/', status=200, tam_conteudo=6245),
 Row(host='unicomp6.unicomp.net', timestamp='01/Jul/1995:00:00:06 -0400', caminho='/shuttle/countdown/', status=200, tam_conteudo=3985),
 Row(host='199.120.110.21', timestamp='01/Jul/1995:00:00:09 -0400', caminho='/shuttle/missions/sts-73/mission-sts-73.html', status=200, tam_conteudo=4085),
 Row(host='burger.letters.com', timestamp='01/Jul/1995:00:00:11 -0400', caminho='/shuttle/countdown/liftoff.html', status=304, tam_conteudo=0),
 Row(host='199.120.110.21', timestamp='01/Jul/1995:00:00:11 -0400', caminho='/shuttle/missions/sts-73/sts-73-patch-small.gif', status=200, tam_conteudo=4179)]

In [14]:
#Verificando se há valores nulos
spark.sql("SELECT COUNT(*) FROM logs").show()

+--------+
|count(1)|
+--------+
| 3461613|
+--------+



In [15]:
#Verifica quais colunas tem valores nulos
spark.sql("""
SELECT 
COUNT(*)-COUNT(host) As host,
COUNT(*)-COUNT(timestamp) As timestamp,
COUNT(*)-COUNT(caminho) As caminho,
COUNT(*)-COUNT(status) As status,
COUNT(*)-COUNT(tam_conteudo) As tam_conteudo
FROM logs
""").show()

+----+---------+-------+------+------------+
|host|timestamp|caminho|status|tam_conteudo|
+----+---------+-------+------+------------+
|   0|        0|      0|     1|       33905|
+----+---------+-------+------+------------+



In [16]:
#subistituindo os valores nulos por 0
df_logs_atual= df_logs.na.fill({'tam_conteudo': 0, 'status':0})

In [17]:
df_logs_atual.createOrReplaceTempView("logs")
spark.sql("SELECT * FROM logs").take(5)

[Row(host='199.72.81.55', timestamp='01/Jul/1995:00:00:01 -0400', caminho='/history/apollo/', status=200, tam_conteudo=6245),
 Row(host='unicomp6.unicomp.net', timestamp='01/Jul/1995:00:00:06 -0400', caminho='/shuttle/countdown/', status=200, tam_conteudo=3985),
 Row(host='199.120.110.21', timestamp='01/Jul/1995:00:00:09 -0400', caminho='/shuttle/missions/sts-73/mission-sts-73.html', status=200, tam_conteudo=4085),
 Row(host='burger.letters.com', timestamp='01/Jul/1995:00:00:11 -0400', caminho='/shuttle/countdown/liftoff.html', status=304, tam_conteudo=0),
 Row(host='199.120.110.21', timestamp='01/Jul/1995:00:00:11 -0400', caminho='/shuttle/missions/sts-73/sts-73-patch-small.gif', status=200, tam_conteudo=4179)]

In [18]:
#Verifica novamente a presença de nulos
spark.sql("""
SELECT 
COUNT(*)-COUNT(host) As host,
COUNT(*)-COUNT(timestamp) As timestamp,
COUNT(*)-COUNT(caminho) As caminho,
COUNT(*)-COUNT(status) As status,
COUNT(*)-COUNT(tam_conteudo) As tam_conteudo
FROM logs
""").show()

+----+---------+-------+------+------------+
|host|timestamp|caminho|status|tam_conteudo|
+----+---------+-------+------+------------+
|   0|        0|      0|     0|           0|
+----+---------+-------+------+------------+



In [21]:
#Cria-se uma função para estrair/converter a data do timestamp (os dias serão importante para responder as questões)
def converte_para_timestamp(date):
        ''' Converte a string de data/time para data'''
        try:
            return datetime.strptime(date.split(':')[0], "%d/%b/%Y").strftime("%Y-%m-%d")
        except Exception as e:
            return 0


converte_data = udf(converte_para_timestamp)
df_final = df_logs_atual.select('*', converte_data (df_logs_atual['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = df_final.count()


In [23]:
#Dataframe após alterações
df_final.printSchema()

root
 |-- host: string (nullable = true)
 |-- caminho: string (nullable = true)
 |-- status: integer (nullable = false)
 |-- tam_conteudo: integer (nullable = false)
 |-- time: timestamp (nullable = true)



In [24]:
df_final.cache() #Cria-se cache pois essa dataframe será usado mais vezes 
df_final.createOrReplaceTempView("logs") #Permite o uso de comandos SQL
df_final.show()

+--------------------+--------------------+------+------------+--------------------+
|                host|             caminho|status|tam_conteudo|                time|
+--------------------+--------------------+------+------------+--------------------+
|        199.72.81.55|    /history/apollo/|   200|        6245|1995-07-01 00:00:...|
|unicomp6.unicomp.net| /shuttle/countdown/|   200|        3985|1995-07-01 00:00:...|
|      199.120.110.21|/shuttle/missions...|   200|        4085|1995-07-01 00:00:...|
|  burger.letters.com|/shuttle/countdow...|   304|           0|1995-07-01 00:00:...|
|      199.120.110.21|/shuttle/missions...|   200|        4179|1995-07-01 00:00:...|
|  burger.letters.com|/images/NASA-logo...|   304|           0|1995-07-01 00:00:...|
|  burger.letters.com|/shuttle/countdow...|   200|           0|1995-07-01 00:00:...|
|     205.212.115.106|/shuttle/countdow...|   200|        3985|1995-07-01 00:00:...|
|         d104.aa.net| /shuttle/countdown/|   200|        3985|19

<h2>1. Número de hosts únicos. </h2>

In [25]:
# Obtem o total de host unicos
total_de_host_unicos = df_final.select('host').distinct().count()
#OU (caso queira usar SQL)
#total_de_host_unicos = spark.sql('SELECT DISTINCT host FROM logs').count()

#IMPRIME RESULTADO
print('Hosts unicos: {0}'.format(total_de_host_unicos))

Hosts unicos: 137933


<h2>2. O total de erros 404. </h2>

In [26]:
#seleciona todos erros 404 
erros_404 = df_final.filter(df_final.status == 404).cache()
#OU (caso queira usar SQL)
#erros_404 = spark.sql('SELECT * FROM logs WHERE status = 404').count()

In [27]:
total_404 = erros_404.count()
print('Total de erros 404: {0}'.format(total_404))

Total de erros 404: 20899


<h2>3. Os 5 URLs que mais causaram erro 404.</h2>

In [28]:
#Obtem o top 5 URLs que causam mais problemas
top5_erros_404 = erros_404.groupBy("host").count().sort("count", ascending=False).limit(5)
#OU (caso queira usar SQL)
#top5_erros_404 = spark.sql('SELECT host, count(*) AS count FROM logs WHERE status = 404 GROUP BY host ORDER BY count DESC LIMIT 5')

In [29]:
top5_erros_404.show(truncate=False)

+---------------------------+-----+
|host                       |count|
+---------------------------+-----+
|hoohoo.ncsa.uiuc.edu       |251  |
|piweba3y.prodigy.com       |157  |
|jbiagioni.npt.nuwc.navy.mil|132  |
|piweba1y.prodigy.com       |114  |
|                           |112  |
+---------------------------+-----+



<h2>4. Quantidade de erros 404 por dia. </h2>

In [30]:
#obtem a quantidade por dia dos erros
erros_404_dias = (erros_404.groupBy(F.dayofmonth('time').alias('day')).count().sort("day"))
#OU (caso queira usar SQL)
#erros_404_dias = spark.sql('SELECT DAY(time) AS day, count(*) AS count FROM logs WHERE status = 404 GROUP BY DAY(time) ORDER BY DAY(time) ASC')

In [31]:
erros_404_dias.show(31)

+---+-----+
|day|count|
+---+-----+
|  1|  559|
|  2|  291|
|  3|  778|
|  4|  705|
|  5|  733|
|  6| 1013|
|  7| 1107|
|  8|  691|
|  9|  627|
| 10|  713|
| 11|  734|
| 12|  667|
| 13|  748|
| 14|  700|
| 15|  581|
| 16|  516|
| 17|  677|
| 18|  721|
| 19|  848|
| 20|  740|
| 21|  639|
| 22|  480|
| 23|  578|
| 24|  748|
| 25|  876|
| 26|  702|
| 27|  706|
| 28|  504|
| 29|  420|
| 30|  571|
| 31|  526|
+---+-----+



<h2>5. O total de bytes retornados. </h2>

In [34]:
total_bytes = spark.sql('SELECT count(tam_conteudo) AS Qtd_Bytes, sum(tam_conteudo) AS Total_Bytes FROM logs')
#OU (caso queira usar SQL)
#total_bytes = df_final.agg(F.count(df_final['tam_conteudo']).alias('Qtd_Bytes'), F.sum(df_final['tam_conteudo']).alias('Qtd_Bytes'))

In [35]:
total_bytes.show()

+---------+-----------+
|Qtd_Bytes|Total_Bytes|
+---------+-----------+
|  3461613|65524314915|
+---------+-----------+

